## Example: Converting a Segmentation Stack + geff for Use with TrackGardener

This notebook demonstrates how to convert a segmentation stack and a [GEFF](https://github.com/live-image-tracking-tools/geff) graph into a TrackGardener database using the configuration file Fluo-N2DL-HeLa-01_config_geff.yaml.

In this workflow, the segmentation stack provides the precise segmentation masks, while the GEFF file supplies only the graph structure. The GEFF file is expected to specify both `t` (timepoint), `track_id` (track ID) and `segm_id` (segmentation ID) for each cell entry. The notebook verifies that for each timepoint t, there is a corresponding object in the segmentation array (labeled stack) with the matching segm_id. For further details on additional checks, please refer to the documentation.

All signals are computed according to the specifications in the configuration file. Please note that this process may take a considerable amount of time for large datasets.

You can reuse the configuration file shown here with the TrackGardener plugin. Be sure to update the file paths in the configuration to point to your imaging dataset and TrackGardener database. It is recommended to use absolute paths (instead of the relative paths used in this example) to ensure correct file location. If you prefer to use relative paths, they should be specified relative to the location of the configuration file.

In [1]:
import dask.array as da
import yaml

from track_gardener.converters import (
    segm_and_geff_to_TG,
    validate_geff_seg_ids,
)

In [2]:
geff_group_path = "Fluo-N2DL-HeLa-01_sample.geff"
segmentation_path = "Fluo-N2DL-HeLa-01_segm_random.zarr/labels"
config_path = 'Fluo-N2DL-HeLa-01_config_geff.yaml'

In [3]:
p, details = validate_geff_seg_ids(
    geff_group_path = geff_group_path,
    segmentation_path = segmentation_path,
    seg_id_field = "segm_id",
    check_xy_position = True,
    check_extra_segmentations = True)

print(f'Test passed: {p}, details: {details}.')

2025-08-12 20:00:46.255 | INFO     | track_gardener.converters.validator_seg_id:validate_geff_seg_ids:31 - Opening GEFF group from: Fluo-N2DL-HeLa-01_sample.geff
2025-08-12 20:00:46.310 | INFO     | track_gardener.converters.validator_seg_id:validate_geff_seg_ids:52 - Beginning node validation...
2025-08-12 20:00:46.319 | WARNING  | track_gardener.converters.validator_seg_id:validate_geff_seg_ids:85 - XY mismatch for index 204 at frame 0: expected position: (51,83)
2025-08-12 20:00:46.326 | WARNING  | track_gardener.converters.validator_seg_id:validate_geff_seg_ids:85 - XY mismatch for index 204 at frame 1: expected position: (53,81)
2025-08-12 20:00:46.346 | WARNING  | track_gardener.converters.validator_seg_id:validate_geff_seg_ids:85 - XY mismatch for index 207 at frame 4: expected position: (44,62)
2025-08-12 20:00:46.358 | WARNING  | track_gardener.converters.validator_seg_id:validate_geff_seg_ids:85 - XY mismatch for index 207 at frame 6: expected position: (46,61)
2025-08-12 20:

Test passed: False, details: {'node_issues': [(0, 'xy_mismatch'), (1, 'xy_mismatch'), (4, 'xy_mismatch'), (6, 'xy_mismatch'), (7, 'xy_mismatch'), (8, 'xy_mismatch'), (9, 'xy_mismatch'), (10, 'xy_mismatch'), (12, 'xy_mismatch'), (13, 'xy_mismatch'), (14, 'xy_mismatch'), (15, 'xy_mismatch'), (16, 'xy_mismatch'), (17, 'xy_mismatch'), (18, 'xy_mismatch'), (19, 'xy_mismatch'), (20, 'xy_mismatch'), (22, 'xy_mismatch'), (23, 'xy_mismatch'), (24, 'xy_mismatch'), (25, 'xy_mismatch'), (27, 'xy_mismatch'), (28, 'xy_mismatch'), (30, 'xy_mismatch'), (31, 'xy_mismatch'), (33, 'xy_mismatch'), (34, 'xy_mismatch'), (35, 'xy_mismatch'), (36, 'xy_mismatch'), (37, 'xy_mismatch'), (39, 'xy_mismatch'), (40, 'xy_mismatch'), (41, 'xy_mismatch'), (42, 'xy_mismatch'), (43, 'xy_mismatch'), (44, 'xy_mismatch'), (45, 'xy_mismatch'), (46, 'xy_mismatch'), (47, 'xy_mismatch'), (48, 'xy_mismatch'), (49, 'xy_mismatch'), (50, 'xy_mismatch'), (52, 'xy_mismatch'), (53, 'xy_mismatch'), (54, 'xy_mismatch'), (56, 'xy_mismatc

In [4]:
# read in configuration file
with open(config_path) as file:
    config = yaml.safe_load(file)

# get segmentation array
segm_array = da.from_zarr(segmentation_path)

In [5]:
segm_and_geff_to_TG(segm_array, geff_group_path, config)

IndexError: list index out of range

In [ ]:
config["database"]["path"]

'Fluo-N2DL-HeLa-01_conversion_result.db'